<a href="https://colab.research.google.com/github/lorenzomazzante/ML01/blob/main/TD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML01 : TD4 - Performance d'un modèle et Validation-Croisée

Réalisé par Cyprien Gilet et Khaled Belahcene

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import scipy.stats as stats

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

# 1. Implémentation fonctions d'évaluation des performances d'un classifieur

**Question 1.** Dans le fichier *functions.ipynb*, implémenter les différentes fonctions permettant d'évaluer la performance d'un classifieur $\delta\in\Delta$ :
- Risques d'erreur empirique par classe $\hat{R}_k(\delta)$ en fonction d'une fonction de perte $L$ quelconque.
- Risque d'erreur empirique moyen $\hat{r}(\delta)$ en fonction d'une fonction de perte $L$ quelconque.
- Accuracy moyenne $\mathrm{Acc}(\delta)$.
- Accuraces par classe $\mathrm{Acc}_k(\delta)$.
- Matrice de confusion $C(\delta)$.

Le code ci-dessous permet d'importer ces fonctions dans ce Notebook présent.

In [10]:
run functions.ipynb

Y_set = [1, 2, 3]
YR = [1 2 1 3 3 1 1 2]
Yhat = [1 1 1 2 3 3 1 2]
L = [[0. 1. 4.]
 [1. 0. 1.]
 [4. 1. 0.]]
Risque d'erreur empirique moyen : 0.75
Accuracy globale : 0.625
Accuracy par classe : [0.75 0.5  0.5 ]
Matrice de confusion: 
 [[0.75 0.   0.25]
 [0.5  0.5  0.  ]
 [0.   0.5  0.5 ]]
Risques d'erreurs par classe : [1.  0.5 0.5]


# 2. Validation croisée KNN sur base de données Diabete_UCI

Dans cette section nous souhaitons appentre le classifieur KNN à partir d'une validation croisée sur la base de données Diabete UCI. Nous considèrerons la fonction de perte $L_{0\text{-}1}$.

Les cellules suivantes permettent d'importer la base de données.

In [12]:
df = pd.read_csv('diabetes_UCI.csv')
print(df.columns.values)
print('df =', df)
print('df_Info =', df.info())

df = df.reset_index()
df.drop(['index'], axis='columns', inplace=True)

['Pregnancies' 'Glucose' 'BloodPressure' 'SkinThickness' 'Insulin' 'BMI'
 'DiabetesPedigreeFunction' 'Age' 'Outcome']
df =      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        

In [13]:

YRdf = pd.DataFrame(df['Outcome'])
print('np.unique(YRdf) =', np.unique(YRdf))
YR = np.array(YRdf).ravel()+1
print('np.unique(YR) =', np.unique(YR))

Xdf = df.copy()
Xdf.drop(['Outcome'], axis='columns', inplace=True)
#Xdf.info()

K = int(np.max(YR))
X = np.array(Xdf)


np.unique(YRdf) = [0 1]
np.unique(YR) = [1 2]


**Question 2.1.** Implémenter la fonction de perte $L_{0\text{-}1}$.

In [22]:
# RÉPONSE À LA QUESTION 2.1:

K = len(np.unique(YR))
L = np.ones((K, K))

for i in range (K):
    L[i, i] = 0

print('L: \n', L)

L: 
 [[0. 1.]
 [1. 0.]]


**Question 2.2.** Créer une procédure de validation croisée permettant selectionner le nombre de voisins optimal de sorte à ce que l'on obtienne une erreur de généralisation satisfaisante. On pourra s'inspirer de la structure suivante :

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [ ]:
# RÉPONSE À LA QUESTION 2.2:
np.random.seed(407)



# 2. Validation croisée sur base de données Abalones

**Question 2.** Comparer à partir d'une validation croisée sur la base de données Abalones les classifieurs LDA, QDA, Naïve Bayes, en considérant la fonction de perte quadratique. Les cellules suivantes permettent d'importer la base de données.

In [25]:
### Load data
from scipy.io import loadmat
data = loadmat('Datasets/abalone_data.mat')
data.keys()

K = int(data['K'][0][0])
print('K =', K)
Xdf = pd.DataFrame(data['X'])
YRdf = pd.DataFrame(data['YR'], columns = ['Labels'])

FileNotFoundError: [Errno 2] No such file or directory: 'abalone_data.mat'

In [ ]:
Xdf

,0,1,2,3,4,5,6,7
0,3.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,3.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,1.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,3.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,2.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,1.0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,3.0,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,3.0,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,1.0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [ ]:
YRdf

,Labels
0,3
1,2
2,2
3,2
4,2
...,...
4172,3
4173,2
4174,2
4175,2


In [ ]:
# FONCTION DE PERTE QUADRATIQUE:


In [ ]:
# RÉPONSE À LA QUESTION 2:
np.random.seed(407)

